___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___
# Question and Answer Chat Bots

----

------

## Loading the Data

We will be working with the Babi Data Set from Facebook Research.

Full Details: https://research.fb.com/downloads/babi/

- Jason Weston, Antoine Bordes, Sumit Chopra, Tomas Mikolov, Alexander M. Rush,
  "Towards AI-Complete Question Answering: A Set of Prerequisite Toy Tasks",
  http://arxiv.org/abs/1502.05698


In [45]:
import pickle
import numpy as np

In [46]:
%cd /content/drive/MyDrive/MOOCs/UDEMY/nlp/UPDATED_NLP_COURSE/UPDATED_NLP_COURSE/06-Deep-Learning

/content/drive/MyDrive/MOOCs/UDEMY/nlp/UPDATED_NLP_COURSE/UPDATED_NLP_COURSE/06-Deep-Learning


In [47]:
with open("train_qa.txt", "rb") as fp:   # Unpickling
    train_data =  pickle.load(fp)

In [48]:
# |from google.colab import drive
# drive.mount('/content/drive')

In [49]:
with open("test_qa.txt", "rb") as fp:   # Unpickling
    test_data =  pickle.load(fp)

----

## Exploring the Format of the Data

In [50]:
type(test_data)

list

In [51]:
type(train_data)

list

In [52]:
len(test_data)

1000

In [53]:
len(train_data)

10000

In [54]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [55]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [56]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [57]:
train_data[0][2]

'no'

-----

## Setting up Vocabulary of All Words

In [58]:
# Create a set that holds the vocab words
vocab = set()

In [59]:
all_data = test_data + train_data

In [60]:
for story, question , answer in all_data:
    # In case you don't know what a union of sets is:
    # https://www.programiz.com/python-programming/methods/set/union
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [61]:
vocab.add('no')
vocab.add('yes')

In [62]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [63]:
vocab_len = len(vocab) + 1 #we add an extra space to hold a 0 for Keras's pad_sequences

In [64]:
max_story_len = max([len(data[0]) for data in all_data])

In [65]:
max_story_len

156

In [66]:
max_question_len = max([len(data[1]) for data in all_data])

In [67]:
max_question_len

6

## Vectorizing the Data

In [68]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [69]:
# Reserve 0 for pad_sequences
vocab_size = len(vocab) + 1

-----------

In [70]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [71]:
# integer encode sequences of words
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [72]:
tokenizer.word_index

{'.': 35,
 '?': 6,
 'apple': 16,
 'back': 13,
 'bathroom': 4,
 'bedroom': 21,
 'daniel': 17,
 'discarded': 24,
 'down': 29,
 'dropped': 36,
 'football': 5,
 'garden': 3,
 'got': 10,
 'grabbed': 12,
 'hallway': 26,
 'in': 18,
 'is': 2,
 'john': 34,
 'journeyed': 27,
 'kitchen': 25,
 'left': 19,
 'mary': 37,
 'milk': 1,
 'moved': 32,
 'no': 7,
 'office': 23,
 'picked': 28,
 'put': 8,
 'sandra': 20,
 'the': 30,
 'there': 9,
 'to': 15,
 'took': 31,
 'travelled': 22,
 'up': 33,
 'went': 11,
 'yes': 14}

In [73]:
train_story_text = []
train_question_text = []
train_answers = []

for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)

In [74]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [75]:
len(train_story_text)

10000

In [76]:
len(train_story_seq)

10000

In [77]:
# word_index = tokenizer.word_index

### Functionalize Vectorization

In [78]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len,max_question_len=max_question_len):
    '''
    INPUT: 
    
    data: consisting of Stories,Queries,and Answers
    word_index: word index dictionary from tokenizer
    max_story_len: the length of the longest story (used for pad_sequences function)
    max_question_len: length of the longest question (used for pad_sequences function)


    OUTPUT:
    
    Vectorizes the stories,questions, and answers into padded sequences. We first loop for every story, query , and
    answer in the data. Then we convert the raw words to an word index value. Then we append each set to their appropriate
    output list. Then once we have converted the words to numbers, we pad the sequences so they are all of equal length.
    
    Returns this in the form of a tuple (X,Xq,Y) (padded based on max lengths)
    '''
    
    
    # X = STORIES
    X = []
    # Xq = QUERY/QUESTION
    Xq = []
    # Y = CORRECT ANSWER
    Y = []
    
    
    for story, query, answer in data:
        
        # Grab the word index for every word in story
        x = [word_index[word.lower()] for word in story]
        # Grab the word index for every word in query
        xq = [word_index[word.lower()] for word in query]
        
        # Grab the Answers (either Yes/No so we don't need to use list comprehension here)
        # Index 0 is reserved so we're going to use + 1
        y = np.zeros(len(word_index) + 1)
        
        # Now that y is all zeros and we know its just Yes/No , we can use numpy logic to create this assignment
        #
        y[word_index[answer]] = 1
        
        # Append each set of story,query, and answer to their respective holding lists
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    # Finally, pad the sequences based on their max length so the RNN can be trained on uniformly long sequences.
        
    # RETURN TUPLE FOR UNPACKING
    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [79]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [80]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [81]:
inputs_test

array([[ 0,  0,  0, ..., 30, 21, 35],
       [ 0,  0,  0, ..., 30,  3, 35],
       [ 0,  0,  0, ..., 30,  3, 35],
       ...,
       [ 0,  0,  0, ..., 30, 16, 35],
       [ 0,  0,  0, ..., 30,  3, 35],
       [ 0,  0,  0, ..., 16,  9, 35]], dtype=int32)

In [82]:
queries_test

array([[ 2, 34, 18, 30, 25,  6],
       [ 2, 34, 18, 30, 25,  6],
       [ 2, 34, 18, 30,  3,  6],
       ...,
       [ 2, 37, 18, 30, 21,  6],
       [ 2, 20, 18, 30,  3,  6],
       [ 2, 37, 18, 30,  3,  6]], dtype=int32)

In [83]:
answers_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [84]:
sum(answers_test)

array([  0.,   0.,   0.,   0.,   0.,   0.,   0., 503.,   0.,   0.,   0.,
         0.,   0.,   0., 497.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.])

In [85]:
tokenizer.word_index['yes']

14

In [86]:
tokenizer.word_index['no']

7

## Creating the Model

In [87]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout
from keras.layers import add, dot, concatenate
from keras.layers import LSTM

### Placeholders for Inputs

Recall we technically have two inputs, stories and questions. So we need to use placeholders. `Input()` is used to instantiate a Keras tensor.


In [88]:
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

### Building the Networks

To understand why we chose this setup, make sure to read the paper we are using:

* Sainbayar Sukhbaatar, Arthur Szlam, Jason Weston, Rob Fergus,
  "End-To-End Memory Networks",
  http://arxiv.org/abs/1503.08895

## Encoders

### Input Encoder m

In [89]:
# Input gets embedded to a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

# This encoder will output:
# (samples, story_maxlen, embedding_dim)

### Input Encoder c

In [90]:
# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

### Question Encoder

In [91]:
# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=max_question_len))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

### Encode the Sequences

In [92]:
# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

##### Use dot product to compute the match between first input vector seq and the query

In [93]:
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

#### Add this match matrix with the second input vector sequence

In [94]:
# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

#### Concatenate

In [95]:
# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

In [96]:
answer

<KerasTensor: shape=(None, 6, 220) dtype=float32 (created by layer 'concatenate')>

In [97]:
# Reduce with RNN (LSTM)
answer = LSTM(32)(answer)  # (samples, 32)

In [98]:
# Regularization with Dropout
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)

In [99]:
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [100]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 156)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 sequential (Sequential)        (None, None, 64)     2432        ['input_1[0][0]']                
                                                                                                  
 sequential_2 (Sequential)      (None, 6, 64)        2432        ['input_2[0][0]']                
                                                                                              

In [101]:
# train
history = model.fit([inputs_train, queries_train], answers_train,batch_size=32,epochs=120,validation_data=([inputs_test, queries_test], answers_test))

Epoch 1/120
313/313 [==============================] - 9s 18ms/step - loss: 0.9130 - accuracy: 0.4970 - val_loss: 0.6955 - val_accuracy: 0.4970
Epoch 2/120
313/313 [==============================] - 5s 15ms/step - loss: 0.7044 - accuracy: 0.4934 - val_loss: 0.6934 - val_accuracy: 0.4970
Epoch 3/120
313/313 [==============================] - 5s 16ms/step - loss: 0.6960 - accuracy: 0.4972 - val_loss: 0.6950 - val_accuracy: 0.4970
Epoch 4/120
313/313 [==============================] - 5s 15ms/step - loss: 0.6946 - accuracy: 0.5056 - val_loss: 0.6938 - val_accuracy: 0.4970
Epoch 5/120
313/313 [==============================] - 5s 16ms/step - loss: 0.6951 - accuracy: 0.4961 - val_loss: 0.6932 - val_accuracy: 0.5030
Epoch 6/120
313/313 [==============================] - 5s 16ms/step - loss: 0.6948 - accuracy: 0.4981 - val_loss: 0.6932 - val_accuracy: 0.4970
Epoch 7/120
313/313 [==============================] - 5s 15ms/step - loss: 0.6946 - accuracy: 0.4967 - val_loss: 0.6934 - val_accuracy:

### Saving the Model

In [102]:
filename = 'chatbot_120_epochs.h5'
model.save(filename)

## Evaluating the Model

### Plotting Out Training History

In [103]:
import matplotlib.pyplot as plt
%matplotlib inline
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


KeyError: ignored

### Evaluating on Given Test Set

In [ ]:
model.load_weights(filename)
pred_results = model.predict(([inputs_test, queries_test]))

In [ ]:
test_data[0][0]

In [ ]:
story =' '.join(word for word in test_data[0][0])
print(story)

In [ ]:
query = ' '.join(word for word in test_data[0][1])
print(query)

In [ ]:
print("True Test Answer from Data is:",test_data[0][2])

In [ ]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

## Writing Your Own Stories and Questions

Remember you can only use words from the existing vocab

In [ ]:
vocab

In [ ]:
# Note the whitespace of the periods
my_story = "John left the kitchen . Sandra dropped the football in the garden ."
my_story.split()

In [ ]:
my_question = "Is the football in the garden ?"

In [ ]:
my_question.split()

In [ ]:
mydata = [(my_story.split(),my_question.split(),'yes')]

In [ ]:
my_story,my_ques,my_ans = vectorize_stories(mydata)

In [ ]:
pred_results = model.predict(([ my_story, my_ques]))

In [ ]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

# Great Job!